# Solvent Similarity Weighting

**Problem**: The CV-LB gap is structural (R²=0.97). Our CV (0.008194) is 4.2x better than target LB (0.0347), but doesn't translate.

**Hypothesis**: Weight training samples by similarity to test solvent to improve generalization.

**Rationale**:
- Research shows sample weighting based on similarity to test distribution can dramatically improve generalization
- This directly addresses the distribution shift problem
- May change the CV-LB relationship

**Baseline**: exp_035 CV 0.008194, LB 0.0877 (predicted)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask]), solvent  # Also yield test solvent

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask]), (row["SOLVENT A NAME"], row["SOLVENT B NAME"])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Solvent similarity computation
def compute_solvent_similarity(train_solvents, test_solvent, sigma=1.0):
    """Compute similarity weights based on Spange descriptors.
    
    Args:
        train_solvents: Series of solvent names for training samples
        test_solvent: Name of test solvent
        sigma: RBF kernel bandwidth
    
    Returns:
        weights: Array of weights for each training sample
    """
    # Get unique solvents and their features
    unique_train_solvents = train_solvents.unique()
    
    # Get test solvent features
    test_features = SPANGE_DF.loc[test_solvent].values.astype(float)
    
    # Compute similarity for each unique solvent
    solvent_similarities = {}
    for solvent in unique_train_solvents:
        train_features = SPANGE_DF.loc[solvent].values.astype(float)
        # Euclidean distance
        dist = np.sqrt(np.sum((train_features - test_features) ** 2))
        # RBF kernel
        similarity = np.exp(-dist ** 2 / (2 * sigma ** 2))
        solvent_similarities[solvent] = similarity
    
    # Map to all training samples
    weights = train_solvents.map(solvent_similarities).values
    
    # Normalize to have mean 1 (so total weight equals number of samples)
    weights = weights / weights.mean()
    
    return weights

# Test the function
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent
weights = compute_solvent_similarity(X_single[mask]["SOLVENT NAME"], test_solvent, sigma=1.0)
print(f'Test solvent: {test_solvent}')
print(f'Weights range: [{weights.min():.4f}, {weights.max():.4f}]')
print(f'Weights mean: {weights.mean():.4f}')

Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Weights range: [0.0000, 16.7297]
Weights mean: 1.0000


In [5]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [6]:
# Weighted Huber Loss with sample weights
class WeightedHuberLoss(nn.Module):
    def __init__(self, target_weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.target_weights = torch.tensor(target_weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target, sample_weights=None):
        huber_loss = self.huber(pred, target)
        # Apply target weights
        weighted_loss = huber_loss * self.target_weights.to(pred.device)
        # Apply sample weights if provided
        if sample_weights is not None:
            weighted_loss = weighted_loss * sample_weights.unsqueeze(1).to(pred.device)
        return weighted_loss.mean()

# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP components defined')

MLP components defined


In [7]:
# Similarity-Weighted MLP Ensemble
class SimilarityWeightedMLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', target_weights=[1.0, 1.0, 2.0], sigma=1.0):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.target_weights = target_weights
        self.sigma = sigma
        self.featurizer = FullFeaturizer(mixed=(data=='full'))
        self.models = []
        self.test_solvent = None

    def train_model(self, X_train, y_train, test_solvent=None):
        self.test_solvent = test_solvent
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        # Compute similarity weights
        if test_solvent is not None and self.data_type == 'single':
            sample_weights = compute_solvent_similarity(X_train["SOLVENT NAME"], test_solvent, self.sigma)
            sample_weights = torch.tensor(sample_weights)
        else:
            sample_weights = torch.ones(len(X_train))
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
            sample_weights = torch.cat([sample_weights, sample_weights], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            model = MLPModelInternal(input_dim, self.hidden_dims).to(device).double()
            optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
            criterion = WeightedHuberLoss(self.target_weights)
            
            dataset = TensorDataset(X_all, y_all, sample_weights)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(200):
                for X_batch, y_batch, w_batch in loader:
                    X_batch, y_batch, w_batch = X_batch.to(device), y_batch.to(device), w_batch.to(device)
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch, w_batch)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize_torch(X_test, flip=False).to(device)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                pred = model(X_std)
                if self.data_type == 'full':
                    pred_flip = model(X_flip)
                    pred = (pred + pred_flip) / 2
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('SimilarityWeightedMLPEnsemble defined')

SimilarityWeightedMLPEnsemble defined


In [8]:
# LightGBM Wrapper with sample weights
class SimilarityWeightedLGBM:
    def __init__(self, data='single', sigma=1.0):
        self.data_type = data
        self.sigma = sigma
        self.featurizer = FullFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, test_solvent=None):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        # Compute similarity weights
        if test_solvent is not None and self.data_type == 'single':
            sample_weights = compute_solvent_similarity(X_train["SOLVENT NAME"], test_solvent, self.sigma)
        else:
            sample_weights = np.ones(len(X_train))
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
            sample_weights = np.concatenate([sample_weights, sample_weights])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i], weight=sample_weights)
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_std)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('SimilarityWeightedLGBM defined')

SimilarityWeightedLGBM defined


In [9]:
# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

# GP Wrapper (unchanged - GP doesn't support sample weights easily)
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SimpleFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, test_solvent=None):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
            X_flip_scaled = self.scaler.transform(X_flip)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip_scaled)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('GP and Simple Featurizer defined')

GP and Simple Featurizer defined


In [10]:
# Similarity-Weighted Ensemble
class SimilarityWeightedEnsemble:
    def __init__(self, data='single', sigma=1.0):
        self.data_type = data
        self.sigma = sigma
        self.gp = GPWrapper(data=data)
        self.mlp = SimilarityWeightedMLPEnsemble(hidden_dims=[32, 16], n_models=5, data=data, 
                                                  target_weights=[1.0, 1.0, 2.0], sigma=sigma)
        self.lgbm = SimilarityWeightedLGBM(data=data, sigma=sigma)
        # Weights: GP 0.15, MLP 0.55, LGBM 0.3 (same as exp_035)
        self.weights = {'gp': 0.15, 'mlp': 0.55, 'lgbm': 0.3}

    def train_model(self, X_train, y_train, test_solvent=None):
        self.gp.train_model(X_train, y_train, test_solvent)
        self.mlp.train_model(X_train, y_train, test_solvent)
        self.lgbm.train_model(X_train, y_train, test_solvent)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('SimilarityWeightedEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3) with similarity weighting')

SimilarityWeightedEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3) with similarity weighting


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

# Modified to get test solvent
all_predictions = []
for fold_idx, solvent in enumerate(sorted(X["SOLVENT NAME"].unique())):
    mask = X["SOLVENT NAME"] != solvent
    train_X, train_Y = X[mask], Y[mask]
    test_X, test_Y = X[~mask], Y[~mask]

    model = SimilarityWeightedEnsemble(data='single', sigma=1.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, test_solvent=solvent)  # Pass test solvent

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
    
    if fold_idx == 0:
        print(f'Fold 0 ({solvent}) completed')

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)
print(f'Single solvent: {len(submission_single_solvent)} predictions')

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol) completed


Single solvent: 656 predictions


In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

# Modified to get test ramp
ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
all_predictions = []

for fold_idx, (_, row) in enumerate(ramps.iterrows()):
    mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
    train_X, train_Y = X[mask], Y[mask]
    test_X, test_Y = X[~mask], Y[~mask]
    test_ramp = (row["SOLVENT A NAME"], row["SOLVENT B NAME"])

    model = SimilarityWeightedEnsemble(data='full', sigma=1.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, test_solvent=test_ramp)  # Pass test ramp (not used for full)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)
print(f'Full data: {len(submission_full_data)} predictions')

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

Full data: 1227 predictions


In [13]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [14]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_035 baseline (no similarity weighting): CV 0.008194')
print(f'This (with similarity weighting): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_035!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_035')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.031267 (n=656)
Full Data MSE: 0.023639 (n=1227)
Overall MSE: 0.026296

exp_035 baseline (no similarity weighting): CV 0.008194
This (with similarity weighting): CV 0.026296

✗ WORSE: 220.92% worse than exp_035
